In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import linregress
import hvplot.pandas

# Fetch S&P 500 monthly data for the last 60 years
sp500 = yf.download('^GSPC', period="30y", interval="1mo")

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calculate RSI
sp500['RSI'] = calculate_rsi(sp500['Adj Close'], window=14)

# Calculate the RSI slope
N = 5
sp500['RSI_Slope'] = sp500['RSI'].rolling(window=N).apply(lambda x: linregress(range(N), x).slope, raw=False)

# Determine Buy/Sell signals
sp500['Signal'] = np.where(sp500['RSI_Slope'] > 0, 'Buy', 'Sell')

# Plot adjustments for size
plot_height = 600
plot_width = 1200

# Plot
rsi_plot = sp500['RSI'].hvplot(line_color='blue', ylabel='RSI', width=plot_width, height=plot_height, title='S&P 500 Monthly RSI & Slope with Buy/Sell Signals (60 Years)')
buy_signals = sp500[sp500['Signal'] == 'Buy']['RSI'].hvplot.scatter(color='green', marker='^', size=100, legend=False)
sell_signals = sp500[sp500['Signal'] == 'Sell']['RSI'].hvplot.scatter(color='red', marker='v', size=100, legend=False)

final_plot = rsi_plot * buy_signals * sell_signals

# Display the plot
final_plot


[*********************100%%**********************]  1 of 1 completed


:Overlay
   .Curve.RSI      :Curve   [Date]   (RSI)
   .Scatter.RSI.I  :Scatter   [Date]   (RSI)
   .Scatter.RSI.II :Scatter   [Date]   (RSI)